In [1]:
import sys
sys.path.append('scripts')

import pandas as pd
import warnings

# Suppress all UserWarnings
warnings.filterwarnings("ignore", category=UserWarning)


In [2]:
%load_ext autoreload
%autoreload 2
from utils import *
from key_metrics import *

c:\Users\kenia\anaconda3\envs\cobre\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

At the moment of loading data automatically is generated an express and basic EDA analysis

In [3]:
orders_df = load_table('orders', profiling=True)
order_payments_df = load_table('order_payments', profiling=True)
customers_df = load_table('customers', profiling=True)
products_df = load_table('products', profiling=True)
order_items_df = load_table('order_items', profiling=True)
order_reviews_df = load_table('order_reviews', profiling=True)
product_category_name_translation_df = load_table('product_category_name_translation', profiling=True)
sellers_df = load_table('sellers', profiling=True)
geolocation_df = load_table('geolocation', profiling=True)
leads_qualified_df = load_table('leads_qualified', profiling=True)
leads_closed_df = load_table('leads_closed', profiling=True)
# customer = customer_view()

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 212.91it/s]


In [14]:
# basic preprocessing
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
orders_df['order_approved_at'] = pd.to_datetime(orders_df['order_approved_at'])
orders_df['order_delivered_carrier_date'] = pd.to_datetime(orders_df['order_delivered_carrier_date'])
orders_df['order_delivered_customer_date'] = pd.to_datetime(orders_df['order_delivered_customer_date'])
orders_df['order_estimated_delivery_date'] = pd.to_datetime(orders_df['order_estimated_delivery_date'])

order_items_df['shipping_limit_date'] = pd.to_datetime(order_items_df['shipping_limit_date'])

order_reviews_df['review_creation_date'] = pd.to_datetime(order_reviews_df['review_creation_date'])
order_reviews_df['review_answer_timestamp'] = pd.to_datetime(order_reviews_df['review_answer_timestamp'])

leads_qualified_df['first_contact_date'] = pd.to_datetime(leads_qualified_df['first_contact_date'])
leads_closed_df['won_date'] = pd.to_datetime(leads_closed_df['won_date'])

In [5]:
explore_dataframe(order_payments_df, 'order_payments', detailed=False)
explore_dataframe(orders_df, 'orders', detailed=False)
explore_dataframe(order_items_df, 'order_items', detailed=False)
explore_dataframe(order_reviews_df, 'order_reviews', detailed=False)
explore_dataframe(products_df, 'products', detailed=False)
explore_dataframe(product_category_name_translation_df, 'product_category_name_translation', detailed=False)
explore_dataframe(sellers_df, 'sellers', detailed=False)
explore_dataframe(geolocation_df, 'geolocation', detailed=False)
explore_dataframe(leads_qualified_df, 'leads_qualified', detailed=False)
explore_dataframe(leads_closed_df, 'leads_closed', detailed=False)



DATAFRAME EXPLORATION REPORT - ORDER_PAYMENTS

Dataset shape: 103886 rows × 5 columns
Memory usage: 17.81 MB

────────────────────────────────────────────────────────────
CATEGORICAL COLUMNS
────────────────────────────────────────────────────────────
Found 1 categorical columns suitable for analysis:
  • payment_type
CATEGORICAL COLUMNS ANALYSIS

──────────────────────────────────────────────────
Column: payment_type
├─ Unique values: 5
├─ Null values: 0 (0.0%)
└─ Fill rate: 100.0%

Value distribution:
  credit_card         :  76795 ( 73.9%) ████████████████████████████████████
  boleto              :  19784 ( 19.0%) █████████
  voucher             :   5775 (  5.6%) ██
  debit_card          :   1529 (  1.5%) 
  not_defined         :      3 (  0.0%) 

Total categorical columns analyzed: 1
Total categorical columns excluded: 0

DATAFRAME EXPLORATION REPORT - ORDERS

Dataset shape: 99441 rows × 8 columns
Memory usage: 58.93 MB

───────────────────────────────────────────────────────────

['business_segment',
 'lead_type',
 'lead_behaviour_profile',
 'average_stock',
 'business_type']

In [11]:
product_category_name_translation_df.product_category_name_english.unique()

array(['health_beauty', 'computers_accessories', 'auto', 'bed_bath_table',
       'furniture_decor', 'sports_leisure', 'perfumery', 'housewares',
       'telephony', 'watches_gifts', 'food_drink', 'baby', 'stationery',
       'tablets_printing_image', 'toys', 'fixed_telephony',
       'garden_tools', 'fashion_bags_accessories', 'small_appliances',
       'consoles_games', 'audio', 'fashion_shoes', 'cool_stuff',
       'luggage_accessories', 'air_conditioning',
       'construction_tools_construction',
       'kitchen_dining_laundry_garden_furniture',
       'costruction_tools_garden', 'fashion_male_clothing', 'pet_shop',
       'office_furniture', 'market_place', 'electronics',
       'home_appliances', 'party_supplies', 'home_confort',
       'costruction_tools_tools', 'agro_industry_and_commerce',
       'furniture_mattress_and_upholstery', 'books_technical',
       'home_construction', 'musical_instruments',
       'furniture_living_room', 'construction_tools_lights',
       'indust

## Key Metrics and Visualization

In [7]:
conversion_rate_by_payment_method(orders_df, order_payments_df)

,payment_type,conversion_rate,total_payment_value
1,credit_card,0.769351,12542084.19
0,boleto,0.198952,2869361.27
4,voucher,0.038877,379436.87
2,debit_card,0.015366,217989.79
3,not_defined,0.000030,0.00


In [12]:
average_value_vs_installments(order_payments_df)

,payment_installments,avg_transaction_value
0,0,94.315000
1,1,112.420229
2,2,127.228150
3,3,142.539317
4,4,163.976840
5,5,183.465222
6,6,209.849952
7,7,187.673672
8,8,307.737427
9,9,203.440870


In [15]:
delivery_time_analysis(order_items_df, orders_df)

,product_id,avg_delivery_time_days
26493,cdad9f8ea258557cae046590bd2ba19c,2.000000
24396,bdf5efab9b8b85e6a49603007646f763,2.000000
1136,08eada1514424f37cd0704a5f45c1e07,2.000000
14022,6c8ddb5440080718b5110d82c7ae1faf,2.000000
28853,dff6fc6c464afef2e1838ae9c787f125,2.000000
...,...,...
7345,38841dec90eddf2f43cdfabc48b3831a,97.000000
19571,984a3b9f9bb4c8feb319da951212696e,148.000000
19377,96ea060e41bdecc64e2de00b97068975,530.000000
17494,87b92e06b320e803d334ac23966c80b1,703.333333


In [16]:
frequent_customers_analysis(orders_df, customers_df)

,customer_id,total_orders,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state


In [17]:
temporal_trends_analysis(orders_df)

,order_month,total_orders
0,2016-09,4
1,2016-10,324
2,2016-12,1
3,2017-01,800
4,2017-02,1780
5,2017-03,2682
6,2017-04,2404
7,2017-05,3700
8,2017-06,3245
9,2017-07,4026


In [18]:
customer_satisfaction_analysis(order_reviews_df)

,review_score,total_reviews
4,5,57076
3,4,19098
2,3,8160
1,2,3148
0,1,11393


In [24]:
%reload_ext autoreload

In [27]:
effectiveness_rate, promotions_by_seller, promotions_by_category = promotions_effectiveness_analysis(orders_df, order_payments_df, order_items_df, sellers_df, products_df)
print(f"Overall Effectiveness Rate of Promotions: {effectiveness_rate:.2%}")
display(promotions_by_seller.head(), promotions_by_category.head())

Overall Effectiveness Rate of Promotions: 3.89%


,seller_id,promo_orders_count,total_orders,promo_effectiveness_rate
1017,fa18c782a4866d313cadab567097f8f1,1,1,1.0
1027,fc0b214b59a83615fea981c6424a02ae,1,1,1.0
411,6576fd3e23c88f0e5d4d23f39bba0542,1,1,1.0
175,28d162ab86bb49d719c32f0ee6186eaa,1,1,1.0
139,1fe61aa5494bd3ae92a2c13e067c810f,1,1,1.0


,product_category_name,promo_orders_count,total_orders,promo_effectiveness_rate
41,la_cuisine,2,13,0.153846
55,pc_gamer,1,8,0.125000
35,fashion_roupa_masculina,10,112,0.089286
14,casa_conforto_2,2,24,0.083333
31,fashion_bolsas_e_acessorios,132,1864,0.070815


In [28]:
cross_selling_analysis(order_items_df, products_df)

,num_categories,total_orders
1,0,1389
0,1,96550
2,2,712
3,3,15


In [29]:
location_impact_analysis(customers_df, orders_df)

,customer_state,total_orders
25,SP,41746
18,RJ,12852
10,MG,11635
22,RS,5466
17,PR,5045
23,SC,3637
4,BA,3380
6,DF,2140
7,ES,2033
8,GO,2020


In [30]:
logistics_impact_analysis(order_items_df, orders_df)

,order_status,total_orders
2,delivered,96478
5,shipped,1106
1,canceled,461
3,invoiced,312
4,processing,301
6,unavailable,6
0,approved,2
